# PPD: Programação com CUDA

Hélio - DC/UFSCar - 2024

# Criando códigos para GPUs: *kernels*

Na programação com CUDA, o fluxo principal do programa continua sendo em CPU, mas é possível transferir parte do processamento para a(s) GPU(s), bem como os dados necessários para esses processamentos.

No funcionamento combinado de CPU e GPU, usa-se o termo ***host*** para designar o sistema computacional, e ***device*** para designar a GPU. Assim, fala-se em processamento no *host* e processamento no *device* (dispositivo). Cabe ao código do *host* coordenar as transferência de dados e códigos para o *device*, bem como ativar suas execuções. Os códigos executados no *device* são chamados de ***kernels***.

De maneira geral, a programação com CUDA envolve as seguintes etapas:

* Declarar e alocar variáveis e espaços de memória para dados na RAM (*host memory*) (para as atividades executando em CPU);
* Declarar e alocar variáveis necessárias no espaço de endereçamento da GPU (*device memory*), para os processamentos em GPU;
* Inicializar dados no *host*;
* Transferir dados do *host* para o *device*;
  <br>[ ou Inicializar dados direteramente no *device*; ]
* Executar um ou mais *kernels* no *device*;
* Transferir os resultados da memória do *device* para a memória do *host*.

A declaração e a ativação de códigos para execução em GPU ocorrem dentro do próprio programa. Cada código de GPU (*kernel*) é declarado como uma função que tem o prefixo **\_\_global__**. Isso indica ao compilador para gerar código para esta função usando os recursos da arquitetura da GPU.

Variáveis declaradas dentro de uma função de *kernel* serão alocadas automaticamente na área de memória do *device*. Essa alocação pode ocorrer em registradores dos processadores da GPU ou em posições de memória, o que é feito pela própria plataforma CUDA.

Na ativação da função do *kernel*, especifica-se também a organizacão lógica das *threads* que serão utilizadas na execução deste código. Assim, o **número de instâncias** que irão executar o *kernel* especificado é determinado pelo **número de *threads*** selecionadas na ativação do *kernel*.

<!--
```
// Kernel definition
__global__ void VecAdd(float* A, float* B, float* C)
{
    int i = threadIdx.x;
    C[i] = A[i] + B[i];
}

int main()
{
    ...
    // Kernel invocation with N threads
    VecAdd<<<1, N>>>(A, B, C);
    ...
}
```
-->

<br>

A ativação do código de um *kernel* em GPU é feita de forma semelhante a uma chamada de função no programa da CPU. O uso dos caracteres ""<<< , >>>"" após o nome da função serve para indicar o número e a organização lógica dos processadores da GPU. Parâmetros da função do *kernel* são especificados depois dessas informações.  

<br>

Considerando o código resultante da compilação de um programa CUDA, a ativação de um *kernel* implica que o código binário gerado para os comandos em alto nível de uma função **__global__** contenha instruções do repertório dos processadores da GPU, bem como outras chamadas ao sistema de controle da GPU para transferência de código para o dispositivo e ativação desses códigos com a composição da grade de *threads* desejada.


# Sobre o código gerado para execução na GPU

Além das funções definidas para execução como *kernels* na GPU, que contêm o prefixo \_\_global__, outras funções do programa que venham a ser solicitadas pelos *kernels* também precisam ser compiladas tendo em vista o código binário do dispositivo.

Para tanto, é preciso introduzir no código de alto nível dessas funções o prefixo ***\_\_device\__***, que indica ao compilador a arquitetura alvo para a geração do código binário.

Mas o que mais pode ser utilizado na criação das funções *kernels*?

Chamadas de sistema, como a leitura e a escrita em arquivos, por exemplo, não têm sido serem realizadas na GPU.

Já funções matemáticas podem ser úteis, mas é claro que são necessárias outras implementações dessas funções específicas para GPUs, ao invés de tentarmos ligar no código a biblioteca padrão **libm**.

De maneira relacionada, o uso da função de impressão no terminal (*stdout*) também tem uma versão específica para GPU.

<br>

A biblioteca com as funções de suporte aos programas é chamada de [**CUDA *runtime***](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#cuda-runtime), que pode ser ligada aos programas de forma estática ou dinâmica.

## CUDA Runtime

    The runtime is implemented in the cudart library, which is linked to the
    application, either statically via cudart.lib or libcudart.a, or
    dynamically via cudart.dll or libcudart.so.

  https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#cuda-runtime


## Formatted output

Pareceu estranho o uso da função ***printf()*** dentro da função (*kernel*) que executa na GPU (*device*)?

Pois é, como indicado no manual [[1](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#formatted-output)], há uma versão específica desta função para ser executada na GPU. Assim como na versão original, esta versão para GPU realiza a formação dos parâmetros numa sequência de caracteres para impressão.

Quando usada na função de um *kernel*, todas as *threads* que executarem este *kernel* irão gerar dados para impressão. Assim, é possível que o programador decida por executá-la apenas em *threads* específicas, identificadas pelos respectivos índices.

Com relação ao volume de impressão, a biblioteca define um *buffer* circular de tamanaho limitado.

Já a impressão efetiva dos dados ocorre quando é executada alguma das operações que promovem sincronização, como cudaDeviceSynchronize() e cudaEventSynchronize(), entre outras, além de após a conclusão de operações de transferência de memória, como cudaMemcpy().

[1] https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#formatted-output


## Mathematical Functions

Outra questão a considerar na definição das funções executadas pelos *kernels* é o uso de funções de biblitecas externas, cujos arquivos binários podem conter código para execução no processador apenas.

Para as principais funções matemáticas, contudo, há também versões compiladas também para uso em GPU.

<br>

https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#mathematical-functions-appendix

<br>

*The reference manual lists, along with their description, all the functions of the C/C++ standard library mathematical functions that are supported in device code, as well as all intrinsic functions (that are only supported in device code).*

*Mathematical functions supported in device code do not set the global errno variable, nor report any floating-point exceptions to indicate errors; thus, if error diagnostic mechanisms are required, the user should implement additional screening for inputs and outputs of the functions. The user is responsible for the validity of pointer arguments. The user must not pass uninitialized parameters to the Mathematical functions as this may result in undefined behavior: functions are inlined in the user program and thus are subject to compiler optimizations.*

# Entendendo o código gerado pelo compilador CUDA

A programação paralela com CUDA consiste na criação de um código que executa em CPU e que controla a operação da(s) GPU(s) para a execução de atividades específicas. Afinal, a GPU pode ser vista como um dispositivo conectado ao computador por alguma interface, como PCI-x.

> Como essa interação pode ser um gargalo, há até mecanismos eficientes para a [interligação direta](https://www.nvidia.com/en-us/data-center/nvlink/) entre múltiplas GPUs no mesmo computador.

Pensando no código executado em CPU, nos casos mais simples, suas tarefas basicamente consistem em:

* alocar áreas de memória RAM para os dados;
* ler dados de entrada, comumente de arquivos;
* alocar áreas de memória na GPU;
* transferir dados necessários para a GPU;
* iniciar a execução do código na GPU;
* copiar dados produzidos na GPU para a memória RAM

<br>

  É claro que pode haver variações e repetições dessas etapas, como no caso em que é preciso repetidamente (a) enviar dados para a GPU, (b) ativar o processamento de um kernel na GPU, (c) copiar para a RAM dados produzidos na GPU.

<br>

 No código C com CUDA, as funções destinadas à execução como kernels  em GPU são declaradas com o prefixo ***\_\_global__***. Além disso, também é possível criar funções que serão usadas pelos kernels, também executadas  em GPU. Para isso, essas funções devem ser declaradas com o prefixo ***\_\_device__***



## Gerando código para a GPU

Cabe ao compilador CUDA compilar separadamente as funções destinadas à execução em GPU, gerando o código binário apropriado. Em geral, [nvcc](https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html) gera e inclui este código no programa executável, mas também é possível gerar um arquivo binário separado.

  O código binário apropriado é transferido para a GPU em tempo de execução, estando pronto para as ativações do kernel.

> [ https://docs.nvidia.com/cuda/cuda-binary-utilities/index.html ] <br>
> *A CUDA binary (also referred to as cubin) file is an ELF-formatted file which consists of CUDA executable code sections as well as other sections containing symbols, relocators, debug info, etc. By default, the CUDA compiler driver nvcc embeds cubin files into the host executable file. But they can also be generated separately by using the "-cubin" option of nvcc. cubin files are loaded at run time by the CUDA driver API.*


> *CUDA provides two binary utilities for examining and disassembling cubin files and host executables: cuobjdump and nvdisasm. Basically, cuobjdump accepts both cubin files and host binaries while nvdisasm only accepts cubin files; but nvdisasm provides richer output options.*


<br>


Os exemplos a seguir realizam a compilação do código da GPU apenas e mostram informações do programa resultante.

In [ ]:
# Veja as seções do programa compilado, salvas num arquivo ELF
! nvcc ind.cu -o ind
! echo
! objdump -h -w ind

In [ ]:
# Veja informações do arquivo elf gerado com o código da GPU apenas
# ! nvcc ind.cu -o ind
! nvcc ind.cu -cubin
! ls -l ind*
! echo
! cuobjdump --dump-elf ind.cubin

In [ ]:
# Veja o código executável do arquivo binário da GPU
# ! nvcc ind.cu -o ind
! nvcc ind.cu -cubin
! echo
! nvdisasm --print-code ind.cubin

# Geração de valores aleatórios na GPU

Considerando a complexidade e o custo computacional para a geração de números aleatórios com distribuições apropriadas, há uma biblioteca específica de CUDA que permite a geração de valores na GPU, de forma paralela e eficiente.

Para tanto, contudo, não é preciso gerar uma função de kernel específica, bastando realizar no *host* chamadas às funções de uma biblioteca específica chamada [curand](https://docs.nvidia.com/cuda/curand/index.html).

<br>

https://docs.nvidia.com/cuda/curand/introduction.html

*cuRAND consists of two pieces: a library on the host (CPU) side and a device (GPU) header file. The host-side library is treated like any other CPU library: users include the header file, /include/curand.h, to get function declarations and then link against the library.*

*Random numbers can be generated on the device or on the host CPU. For device generation, calls to the library happen on the host, **but the actual work of random number generation occurs on the device**. **The resulting random numbers are stored in global memory on the device**. Users can then call their own kernels to use the random numbers, or they can copy the random numbers back to the host for further processing. For host CPU generation, all of the work is done on the host, and the random numbers are stored in host memory.*

<br>

O programa a seguir ilustra o uso da biblioteca curand para geração de valores aleatórios na GPU a partir de código executado no *host*.

Também é possível usar chamadas da API em *kernels* e funções compiladas para execução na GPU (__device__).


In [ ]:
%%writefile rand.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand.h>

int
main()
{
  size_t n = 100;
  size_t i;
  curandGenerator_t gen;
  float *devData, *hostData;

  // Aloca espaco para n floats na RAM
  hostData = (float *)malloc(n * sizeof(float));

  // Aloca espaco para n floats na GPU
  cudaMalloc((void **)&devData, n * sizeof(float));

  // Cria um gerador de numeros pseuso-aleatorios
  curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT);

  // Inicia semente de geracao
  curandSetPseudoRandomGeneratorSeed(gen, 1234ULL);

  // Gera n floats na GPU
  curandGenerateUniform(gen, devData, n);

  // Dados gerados na GPU podem ser usados em algum kernel específico, uma
  // vez que os dados gerados são salvos na memória da GPU

  // Copia numeros gerados da GPU para a RAM
  cudaMemcpy(hostData, devData, n * sizeof(float), cudaMemcpyDeviceToHost);

  // Exibe valores gerados
  printf("índice, valor\n");
  for(i = 0; i < n; i++)
    printf("%d, %1.4f\n", (int)i, hostData[i]);
  printf("\n");

  // Libera recursos
  curandDestroyGenerator(gen);
  cudaFree(devData);

  free(hostData);

  return 0;
}

Writing rand.cu


In [ ]:
! if [ ! rand -nt rand.cu ]; then nvcc rand.cu -o rand -lcurand -O3; fi
! ./rand > result.csv
! cat ./result.csv